In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import fuzz
import pickle
import scipy.sparse

In [22]:
sparse_matrix_test = scipy.sparse.load_npz('obj/sparse_matrix.npz')

In [23]:
sparse_matrix_test

<3000x186980 sparse matrix of type '<class 'numpy.float64'>'
	with 13480976 stored elements in Compressed Sparse Row format>

In [24]:
with open('obj/game_to_idx.pkl', 'rb') as fp:
    idx_test = pickle.load(fp)

In [25]:
idx_test

{'Die Macher': 0,
 'Samurai': 1,
 'Acquire': 2,
 'Cathedral': 3,
 'El Caballero': 4,
 'Elfenland': 5,
 'Bohnanza': 6,
 'Ra': 7,
 'Catan': 8,
 'Basari': 9,
 'Cosmic Encounter': 1137,
 'RoboRally': 11,
 'Wacky Wacky West': 12,
 'Magic Realm': 13,
 'Age of Renaissance': 14,
 'Supremacy: The Game of the Superpowers': 15,
 'Illuminati (Second Edition)': 16,
 'Dark Tower': 17,
 "Can't Stop": 18,
 'Tigris & Euphrates': 19,
 'David & Goliath': 20,
 'Perudo': 21,
 'Medici': 22,
 'Chinatown': 23,
 'Mamma Mia!': 24,
 'Lost Cities': 25,
 'Ricochet Robots': 26,
 'Tikal': 27,
 'Kings & Things': 28,
 'Giganten': 29,
 'Vinci': 30,
 'Löwenherz': 31,
 'Big City': 32,
 'Civilization': 33,
 'Verräter': 34,
 'Show Manager': 35,
 'Apples to Apples': 36,
 'Falling': 37,
 'Torres': 38,
 'Paths of Glory': 39,
 'El Grande': 40,
 'Union Pacific': 41,
 'Igel Ärgern': 42,
 'Conquest of the Empire': 744,
 'Axis & Allies': 629,
 'Fortress America': 45,
 'Detroit-Cleveland Grand Prix': 46,
 'Titan': 47,
 'Colossal Ar

In [126]:
%env JOBLIB_TEMP_FOLDER=/tmp
# define model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-2)

env: JOBLIB_TEMP_FOLDER=/tmp


In [145]:
def fuzzy_matching(mapper, fav_game, verbose=True):
    """
    return the closest match via fuzzy ratio. If no match found, return None
    
    Parameters
    ----------    
    mapper: dict, map movie title name to index of the movie in data

    fav_movie: str, name of user input movie
    
    verbose: bool, print log if True

    Return
    ------
    index of the closest match
    """
    match_tuple = []
    # get match
    for name, idx in mapper.items():
        ratio = fuzz.ratio(name.lower(), fav_game.lower())
        if ratio >= 60:
            match_tuple.append((name, idx, ratio))
    # sort
    match_tuple = sorted(match_tuple, key=lambda x: x[2])[::-1]
    if not match_tuple:
        print('Oops! No match is found')
        return
    if verbose:
        print('Found possible matches in our database: {0}\n'.format([x[0] for x in match_tuple]))
    return match_tuple[0][1]



def make_recommendation(model_knn, data, mapper, fav_game, n_recommendations):
    """
    return top n similar movie recommendations based on user's input movie


    Parameters
    ----------
    model_knn: sklearn model, knn model

    data: movie-user matrix

    mapper: dict, map movie title name to index of the movie in data

    fav_movie: str, name of user input movie

    n_recommendations: int, top n recommendations

    Return
    ------
    list of top n similar movie recommendations
    """
    # fit
    model_knn.fit(data)
    # get input movie index
    print('You have input movie:', fav_game)
    idx = fuzzy_matching(mapper, fav_game, verbose=True)
    # inference
    #print('Recommendation system start to make inference')
    #print('......\n')
    distances, indices = model_knn.kneighbors(data[idx], n_neighbors=n_recommendations+1)
    # get list of raw idx of recommendations
    raw_recommends = \
        sorted(list(zip(indices.squeeze().tolist(), distances.squeeze().tolist())), key=lambda x: x[1])[:0:-1]
    # get reverse mapper
    reverse_mapper = {v: k for k, v in mapper.items()}
    # print recommendations
    #print('Recommendations for {}:'.format(fav_game))
    ranks = np.arange(1, len(raw_recommends)+1, 1)
    recs = []
    for idx, dist in raw_recommends:
        recs.append(reverse_mapper[idx])
    res = {'Rank' : ranks, 'Game' : recs}
    return res
    
    


In [146]:
my_favorite = 'Ticket to Ride'

results = make_recommendation(
    model_knn=model_knn,
    data=sparse_matrix_test,
    fav_game=my_favorite,
    mapper=idx_test,
    n_recommendations=10)

You have input movie: Ticket to Ride
Found possible matches in our database: ['Ticket to Ride', 'Ticket to Ride: London', 'Ticket to Ride: Europe', 'Ticket to Ride: Märklin', 'Ticket to Ride: New York', 'Ticket to Ride: USA 1910', 'Ticket to Ride: Europa 1912', 'Ticket to Ride: Switzerland', 'Ticket to Ride: Rails & Sails', 'Ticket to Ride: The Card Game', 'Ticket to Ride: Alvin & Dexter', 'T.I.M.E Stories', 'Ticket to Ride: 10th Anniversary', 'Ticket to Ride: Nordic Countries']



In [147]:
results

{'Rank': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 'Game': ['Splendor',
  'King of Tokyo',
  'Puerto Rico',
  'Power Grid',
  'Small World',
  '7 Wonders',
  'Dominion',
  'Pandemic',
  'Carcassonne',
  'Catan']}

In [150]:
pd.DataFrame(results)

,Rank,Game
0,1,Splendor
1,2,King of Tokyo
2,3,Puerto Rico
3,4,Power Grid
4,5,Small World
5,6,7 Wonders
6,7,Dominion
7,8,Pandemic
8,9,Carcassonne
9,10,Catan


In [107]:
results[0]

['Splendor',
 'King of Tokyo',
 'Puerto Rico',
 'Power Grid',
 'Small World',
 '7 Wonders',
 'Dominion',
 'Pandemic',
 'Carcassonne',
 'Catan']

In [118]:
pd.DataFrame('Rank' = results[1], 'Game' = results[0])

SyntaxError: keyword can't be an expression (<ipython-input-118-3f306efb1725>, line 1)

In [117]:
pd.DataFrame(results[1])

,0
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,9
9,10


In [119]:
type(results[0])

list

In [120]:
type(results[1])

numpy.ndarray

In [99]:
lst2 = np.arange(1,11,1)

In [100]:
lst2

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [102]:
pd.DataFrame(results, lst2)

,0
1,Splendor
2,King of Tokyo
3,Puerto Rico
4,Power Grid
5,Small World
6,7 Wonders
7,Dominion
8,Pandemic
9,Carcassonne
10,Catan


In [148]:
my_favorite = 'KLASK'

results2 = make_recommendation(
    model_knn=model_knn,
    data=sparse_matrix_test,
    fav_game=my_favorite,
    mapper=idx_test,
    n_recommendations=10)

You have input movie: KLASK
Found possible matches in our database: ['KLASK', 'Clans', 'Alias', 'Kalah', 'TAMSK']



In [149]:
results2

{'Rank': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 'Game': ['Welcome To...',
  'Santorini',
  'Patchwork',
  'Magic Maze',
  'Azul',
  "That's Pretty Clever!",
  'The Mind',
  'Kingdomino',
  'Just One',
  'ICECOOL']}

In [151]:
pd.DataFrame(results2)

,Rank,Game
0,1,Welcome To...
1,2,Santorini
2,3,Patchwork
3,4,Magic Maze
4,5,Azul
5,6,That's Pretty Clever!
6,7,The Mind
7,8,Kingdomino
8,9,Just One
9,10,ICECOOL


In [152]:
my_favorite = 'Wits & Wagers'

make_recommendation(
    model_knn=model_knn,
    data=sparse_matrix_test,
    fav_game=my_favorite,
    mapper=idx_test,
    n_recommendations=10)

You have input movie: Wits & Wagers
Found possible matches in our database: ['Wits & Wagers', 'Wits & Wagers Party', 'Wits & Wagers Family', 'Axis & Allies']



{'Rank': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 'Game': ['Telestrations',
  'Diamant',
  'Dominion',
  'Blokus',
  'Apples to Apples',
  'Lost Cities',
  'Ticket to Ride',
  'Say Anything',
  'No Thanks!',
  'For Sale']}

In [153]:
my_favorite = 'Carcassonne'

make_recommendation(
    model_knn=model_knn,
    data=sparse_matrix_test,
    fav_game=my_favorite,
    mapper=idx_test,
    n_recommendations=10)

You have input movie: Carcassonne
Found possible matches in our database: ['Carcassonne', 'Cardcassonne', 'My First Carcassonne', 'Carcassonne Big Box 6', 'Carcassonne: Amazonas', 'Carcassonne Big Box 5', 'Carcassonne Big Box 4', 'Carcassonne: The City', 'Carcassonne: Star Wars', 'Carcassonne: Gold Rush', 'Carcassonne: The River', 'Carcassonne: South Seas', 'Carcassonne: The Castle', 'Carcassonne: The Ferries', 'Carcassonne: The Phantom', 'Carcassonne: Promo Tiles', 'Cacao', 'Charterstone', 'Carcassonne: The River II', 'Carcassonne: King & Scout']



{'Rank': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 'Game': ['Power Grid',
  'Ticket to Ride: Europe',
  'Small World',
  'Agricola',
  'Puerto Rico',
  '7 Wonders',
  'Ticket to Ride',
  'Dominion',
  'Pandemic',
  'Catan']}

In [154]:
my_favorite = 'Harry Potter: Hogwarts Battle'

make_recommendation(
    model_knn=model_knn,
    data=sparse_matrix_test,
    fav_game=my_favorite,
    mapper=idx_test,
    n_recommendations=10)

You have input movie: Harry Potter: Hogwarts Battle
Found possible matches in our database: ['Harry Potter: Hogwarts Battle', 'Harry Potter Trading Card Game']



{'Rank': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 'Game': ['Legendary: A Marvel Deck Building Game',
  'Pandemic Legacy: Season 1',
  'Sagrada',
  'Azul',
  'Pandemic',
  'Codenames',
  'Kingdomino',
  'Disney Villainous',
  'Clank!: A Deck-Building Adventure',
  'Harry Potter: Hogwarts Battle – The Monster Box of Monsters Expansion']}

In [155]:
games = pd.read_csv("games_list.csv")

In [156]:
games

,id,name,nrate,pic_url
0,30549,Pandemic,96289,https://cf.geekdo-images.com/micro/img/0m3-oqB...
1,822,Carcassonne,96272,https://cf.geekdo-images.com/micro/img/z0tTaij...
2,13,Catan,96253,https://cf.geekdo-images.com/micro/img/e0y6Bog...
3,68448,7 Wonders,79916,https://cf.geekdo-images.com/micro/img/h-Ejv31...
4,36218,Dominion,74982,https://cf.geekdo-images.com/micro/img/VYp2s2f...
...,...,...,...,...
2995,241533,Mansions of Madness: Second Edition – Sanctum ...,978,https://cf.geekdo-images.com/micro/img/C4_W4C-...
2996,37235,Agricola Z-Deck,977,https://cf.geekdo-images.com/micro/img/yrYSRQN...
2997,8552,I Go!,976,https://cf.geekdo-images.com/micro/img/08Sp6on...
2998,20542,Advanced Squad Leader: Starter Kit #3,976,https://cf.geekdo-images.com/micro/img/1Z7Phwo...


In [158]:
games_list = games.name.tolist()

In [159]:
games_list

['Pandemic',
 'Carcassonne',
 'Catan',
 '7 Wonders',
 'Dominion',
 'Ticket to Ride',
 'Codenames',
 'Agricola',
 'Puerto Rico',
 'Small World',
 'Terraforming Mars',
 'King of Tokyo',
 'Ticket to Ride: Europe',
 'Splendor',
 'Power Grid',
 '7 Wonders Duel',
 'Love Letter',
 'Scythe',
 'Dixit',
 'Citadels',
 'Azul',
 'Race for the Galaxy',
 'Lords of Waterdeep',
 'The Castles of Burgundy',
 'Stone Age',
 'Patchwork',
 'Munchkin',
 'Forbidden Island',
 'Twilight Struggle',
 'Pandemic Legacy: Season 1',
 'Dead of Winter: A Crossroads Game',
 'Terra Mystica',
 'Gloomhaven',
 'Bohnanza',
 'Hanabi',
 'Arkham Horror',
 'Lost Cities',
 'Betrayal at House on the Hill',
 'The Resistance',
 'Jaipur',
 'Coup',
 'Star Realms',
 'Takenoko',
 'Wingspan',
 'Sushi Go!',
 'Battlestar Galactica: The Board Game',
 'Robinson Crusoe: Adventures on the Cursed Island',
 'Blood Rage',
 'Magic: The Gathering',
 'Five Tribes',
 'Mysterium',
 'Dominion: Intrigue',
 'Risk',
 "Tzolk'in: The Mayan Calendar",
 'Kingd

In [160]:
games_list[5]

'Ticket to Ride'

In [161]:
my_favorite = games_list[5]

make_recommendation(
    model_knn=model_knn,
    data=sparse_matrix_test,
    fav_game=my_favorite,
    mapper=idx_test,
    n_recommendations=10)

You have input movie: Ticket to Ride
Found possible matches in our database: ['Ticket to Ride', 'Ticket to Ride: London', 'Ticket to Ride: Europe', 'Ticket to Ride: Märklin', 'Ticket to Ride: New York', 'Ticket to Ride: USA 1910', 'Ticket to Ride: Europa 1912', 'Ticket to Ride: Switzerland', 'Ticket to Ride: Rails & Sails', 'Ticket to Ride: The Card Game', 'Ticket to Ride: Alvin & Dexter', 'T.I.M.E Stories', 'Ticket to Ride: 10th Anniversary', 'Ticket to Ride: Nordic Countries']



{'Rank': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 'Game': ['Splendor',
  'King of Tokyo',
  'Puerto Rico',
  'Power Grid',
  'Small World',
  '7 Wonders',
  'Dominion',
  'Pandemic',
  'Carcassonne',
  'Catan']}

In [163]:
my_favorite = 'Splendor'

make_recommendation(
    model_knn=model_knn,
    data=sparse_matrix_test,
    fav_game=my_favorite,
    mapper=idx_test,
    n_recommendations=10)

You have input movie: Splendor
Found possible matches in our database: ['Splendor', 'Suspend', 'Endeavor', 'Zendo']



{'Rank': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 'Game': ['Azul',
  'Five Tribes',
  'The Castles of Burgundy',
  'Jaipur',
  '7 Wonders Duel',
  'Patchwork',
  'Love Letter',
  'Pandemic',
  '7 Wonders',
  'Codenames']}